<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=de0df2ded62a0c12e89bb8cc513a962894d3cf2e0c7a0fb2b2796df2194604bf
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.3
    Uninstalling protobuf-6.33.3:
      Successfully uninstalled protobuf-6.33.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-12 09:36:06
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -97.39 K (-0.69%)
Current PnL: -29.01 L (-18.5%)
CY Booked + Current PnL: -14.72 L (-9.39%)
-------------------
Total profit:  1.67 L
Total loss:  -30.68 L
-------------------
Total Booked + Current PnL: 12.26 L (9.44%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.47 L (67.45%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.12,14.39,6.13,21.40,18612.0,38189.0,303615.0,6.07,59.0,X-LC,5.84,14.0,2.05,2.17,21.63,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.90,-21.41,27.37,0.09,21676.0,-21581.0,79196.0,83.78,37.0,M-SC,3.31,253.0,-1.00,0.57,4.05,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.27,3.16,15.57,19.23,24167.0,4754.0,155215.0,21.35,65.0,L-LC,7.51,182.0,0.20,1.11,25.11,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,0.56,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,54.0,M-SC,8.33,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
49,MASFIN,398.61,-1.66,-6.48,30.50,22.05,27949.0,-6345.0,91635.0,-19.15,42.0,H-SC,6.15,168.0,-0.23,0.65,33.20,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,2.83,-41.53,131.35,35.27,87209.0,-47155.0,66394.0,-55.73,46.0,L-SC,4.41,268.0,-0.54,0.47,62.50,XR,NTT,HOTELS
23,DMART,5391.45,2.45,-6.38,38.43,29.60,89802.0,-15924.0,233676.0,-20.12,56.0,X-LC,9.63,10.0,-0.18,1.67,14.41,X40N,ATH,FMCG
36,ICICIPRULI,790.00,1.27,14.87,13.81,30.73,31251.0,29288.0,226293.0,-11.98,75.0,X-MC,6.97,79.0,0.94,1.62,29.53,X40,ATH,INSURANCE
81,VBL,671.64,1.26,0.02,35.64,35.67,112589.0,64.0,315906.0,-11.86,57.0,X-LC,2.30,5.0,0.00,2.26,13.88,X40N,ATH,FMCG
37,IEX,219.00,1.18,-5.00,56.44,48.63,107217.0,-9988.0,189966.0,-36.71,49.0,H-SC,13.81,137.0,-0.09,1.36,6.04,XR,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,518.00,-2.66,-68.28,215.47,0.07,93757.0,-93664.0,43513.0,1494.70,32.0,L-SC,2.09,269.0,-1.00,0.31,8.22,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-2.49,-47.52,231.82,74.15,199057.0,-77738.0,85867.0,-254.71,36.0,H-SC,21.60,145.0,-0.39,0.61,10.07,XR,NTT,CHEMICALS
48,LAOPALA,464.00,-2.19,-41.55,137.95,39.09,105718.0,-54470.0,76635.0,67.64,27.0,H-SC,13.78,155.0,-0.52,0.55,0.00,AR,NTT,CERAMICS
76,TRIDENT,48.00,-2.18,-31.41,87.35,28.51,55276.0,-28974.0,63281.0,-18.80,34.0,M-SC,7.73,226.0,-0.52,0.45,8.28,XR,NTT,TEXTILES
25,FINCABLES,1641.55,-2.13,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,44.0,M-SC,10.11,220.0,-0.04,1.01,5.17,OX40N,ATH,CABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.13,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,44.0,M-SC,10.11,220.0,-0.04,1.01,5.17,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.56,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,54.0,M-SC,8.33,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.77,-18.62,62.23,32.03,66768.0,-24542.0,107293.0,-12.01,62.0,H-SC,6.83,157.0,-0.37,0.77,20.18,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-0.90,-21.41,27.37,0.09,21676.0,-21581.0,79196.0,83.78,37.0,M-SC,3.31,253.0,-1.00,0.57,4.05,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.15,-22.53,46.48,13.47,97096.0,-60768.0,208899.0,-68.12,46.0,H-SC,4.49,159.0,-0.63,1.49,6.22,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,0.27,3.16,15.57,19.23,24167.0,4754.0,155215.0,21.35,65.0,L-LC,7.51,182.0,0.20,1.11,25.11,XY25,ATH,MINING
84,WIPRO,420.00,0.21,7.67,60.00,72.27,109463.0,12990.0,182438.0,-7.44,50.0,M-LC,6.18,101.0,0.12,1.30,14.31,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.21,7.67,60.00,72.27,109463.0,12990.0,182438.0,-7.44,50.0,M-LC,6.18,101.0,0.12,1.30,14.31,XR,NTT,IT
25,FINCABLES,1641.55,-2.13,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,44.0,M-SC,10.11,220.0,-0.04,1.01,5.17,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.40,-8.17,125.12,106.72,141706.0,-10080.0,113256.0,-54.44,33.0,H-SC,7.67,138.0,-0.07,0.81,13.29,AR,ATH,MISC
37,IEX,219.00,1.18,-5.00,56.44,48.63,107217.0,-9988.0,189966.0,-36.71,49.0,H-SC,13.81,137.0,-0.09,1.36,6.04,XR,NTT,MISC
1,ABB,7934.00,-1.63,-8.08,58.36,45.57,140344.0,-21139.0,240480.0,-40.92,41.0,H-MC,4.73,121.0,-0.15,1.72,2.33,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.00,-17.37,34.06,10.78,67637.0,-41737.0,198581.0,-50.96,12.0,X-LC,7.10,1.0,-0.62,1.42,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.49,-28.81,120.81,57.19,280485.0,-93957.0,232170.0,-66.31,21.0,X-MC,9.84,58.0,-0.33,1.66,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,0.29,-12.98,50.10,30.62,103346.0,-30774.0,206280.0,-38.20,27.0,X-MC,16.66,57.0,-0.30,1.47,0.29,X40,ATH,APPARELS
78,UNITDSPR,1644.00,-1.27,-3.47,25.10,20.77,56732.0,-8119.0,226025.0,-11.84,28.0,X-MC,1.34,70.0,-0.14,1.61,3.21,X40N,NTT,BREWERIES
11,BATAINDIA,2096.00,-1.10,-40.92,128.83,35.20,97942.0,-52646.0,76024.0,-0.36,29.0,X-SC,17.45,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-1.27,-3.47,25.10,20.77,56732.0,-8119.0,226025.0,-11.84,28.0,X-MC,1.34,70.0,-0.14,1.61,3.21,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-1.20,-8.71,36.36,24.48,88743.0,-23297.0,244068.0,-10.27,35.0,X-LC,1.57,37.0,-0.26,1.74,41.47,XY24,BTT,FINANCE
65,SIEMENS,4671.50,-1.80,-19.69,56.29,25.51,84123.0,-36650.0,149445.0,-8.36,37.0,H-LC,1.94,50.0,-0.44,1.07,10.13,AR,ATH,ELECTRICAL
81,VBL,671.64,1.26,0.02,35.64,35.67,112589.0,64.0,315906.0,-11.86,57.0,X-LC,2.30,5.0,0.00,2.26,13.88,X40N,ATH,FMCG
73,TCS,4389.97,-0.40,-12.15,37.41,20.71,124302.0,-45967.0,332270.0,-24.27,46.0,X-LC,2.32,3.0,-0.37,2.37,11.02,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.66,-28.89,130.85,64.15,221398.0,-68751.0,169200.0,-58.50,31.0,X-SC,6.42,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.0,-1.49,-28.81,120.81,57.19,280485.0,-93957.0,232170.0,-66.31,21.0,X-MC,9.84,58.0,-0.33,1.66,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.0,-1.10,-40.92,128.83,35.20,97942.0,-52646.0,76024.0,-0.36,29.0,X-SC,17.45,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
8,BAJAJHFL,181.5,-0.19,-20.04,94.45,55.47,183725.0,-48765.0,194521.0,-29.11,32.0,X-MC,15.71,63.0,-0.27,1.39,0.0,X40N,ATH,FINANCE
58,RELAXO,1176.0,-1.79,-49.08,199.12,52.31,147174.0,-71248.0,73912.0,-45.79,34.0,X-SC,7.94,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.65,-29.38,102.26,42.84,46947.0,-19097.0,45909.0,-53.72,45.0,X-SC,26.56,83.0,-0.41,0.33,3.86,XY24,NTT,MISC
58,RELAXO,1176.00,-1.79,-49.08,199.12,52.31,147174.0,-71248.0,73912.0,-45.79,34.0,X-SC,7.94,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.10,-40.92,128.83,35.20,97942.0,-52646.0,76024.0,-0.36,29.0,X-SC,17.45,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.63,-2.47,27.23,24.08,33713.0,-3142.0,123808.0,-10.30,44.0,X-SC,6.23,91.0,-0.09,0.88,16.66,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.46,-18.11,75.01,43.32,100048.0,-29492.0,133380.0,-29.66,41.0,X-SC,7.94,90.0,-0.29,0.95,2.41,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.00,-17.37,34.06,10.78,67637.0,-41737.0,198581.0,-50.96,12.0,X-LC,7.10,1.0,-0.62,1.42,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.49,-27.77,70.26,22.98,114637.0,-62737.0,163161.0,-25.52,40.0,X-LC,4.07,2.0,-0.55,1.17,2.62,XY24,NTT,AUTO
73,TCS,4389.97,-0.40,-12.15,37.41,20.71,124302.0,-45967.0,332270.0,-24.27,46.0,X-LC,2.32,3.0,-0.37,2.37,11.02,X40,ATH,IT
81,VBL,671.64,1.26,0.02,35.64,35.67,112589.0,64.0,315906.0,-11.86,57.0,X-LC,2.30,5.0,0.00,2.26,13.88,X40N,ATH,FMCG
41,INFY,1972.00,-1.00,8.78,23.40,34.24,79648.0,27486.0,340374.0,-15.29,46.0,X-LC,7.64,6.0,0.35,2.43,17.79,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.70,-34.41,102.61,32.89,51961.0,-26568.0,50639.0,-725.26,59.0,L-MC,7.81,259.0,-0.51,0.36,39.46,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.56,-14.26,16.65,0.02,24920.0,-24891.0,149668.0,128.24,54.0,M-SC,8.33,240.0,-1.00,1.07,30.19,OX40N,NTT,PAINTS
17,COALINDIA,484.83,0.27,3.16,15.57,19.23,24167.0,4754.0,155215.0,21.35,65.0,L-LC,7.51,182.0,0.20,1.11,25.11,XY25,ATH,MINING
82,VOLTAS,1856.00,-0.18,14.54,26.76,45.20,58773.0,27888.0,219630.0,3.34,61.0,X-MC,5.18,80.0,0.47,1.57,22.33,XY25,NTT,AC
36,ICICIPRULI,790.00,1.27,14.87,13.81,30.73,31251.0,29288.0,226293.0,-11.98,75.0,X-MC,6.97,79.0,0.94,1.62,29.53,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.42,-32.56,67.42,12.90,99341.0,-71149.0,147347.0,117.55,54.0,H-SC,10.07,173.0,-0.72,1.05,22.52,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.77,-18.62,62.23,32.03,66768.0,-24542.0,107293.0,-12.01,62.0,H-SC,6.83,157.0,-0.37,0.77,20.18,OX40N,NTT,IT
17,COALINDIA,484.83,0.27,3.16,15.57,19.23,24167.0,4754.0,155215.0,21.35,65.0,L-LC,7.51,182.0,0.20,1.11,25.11,XY25,ATH,MINING
36,ICICIPRULI,790.00,1.27,14.87,13.81,30.73,31251.0,29288.0,226293.0,-11.98,75.0,X-MC,6.97,79.0,0.94,1.62,29.53,X40,ATH,INSURANCE
82,VOLTAS,1856.00,-0.18,14.54,26.76,45.20,58773.0,27888.0,219630.0,3.34,61.0,X-MC,5.18,80.0,0.47,1.57,22.33,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,44.59
2,50,76.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.52
MC    30.78
LC    26.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.28
X40      24.05
X40N     12.87
XY25     10.80
XR        8.92
AR        8.53
OX40N     7.63
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.41
X-LC    23.25
H-SC    22.71
M-SC    11.23
X-SC     7.80
H-MC     4.59
M-MC     1.42
M-LC     1.30
L-LC     1.11
H-LC     1.07
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-9.04,44.82
IT,12.89,-18.33,78.02
FINANCE,11.01,-15.56,64.06
MISC,7.06,-31.55,85.37
ELECTRICAL,5.92,-13.24,54.72
PAINTS,5.81,-13.41,30.25
INSURANCE,4.78,-1.42,36.12
PHARMA,3.92,-6.76,40.36
AUTO,2.78,-35.01,81.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3292787.0,21
AR,1379266.0,10
XR,1240688.0,12
X40,1149503.0,15
X40N,940093.0,9
OX40N,724852.0,10
XY25,431853.0,7
SR,287768.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3615324.0,24
X-MC,1613998.0,16
M-SC,1488939.0,15
X-LC,968453.0,13
X-SC,826414.0,8
H-MC,436197.0,3
L-SC,180966.0,2
M-LC,109463.0,1
H-LC,84123.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270902.0      6
           AR         932856.0      5
M-SC       XY24       856650.0      6
H-SC       XR         805143.0      7
X-MC       X40        521278.0      7
           XY24       452950.0      3
X-LC       X40        427665.0      6
X-MC       X40N       413906.0      4
X-SC       X40N       323796.0      3
H-SC       OX40N      318655.0      4
M-SC       OX40N      312440.0      5
X-SC       XY24       302058.0      3
H-SC       SR         287768.0      2
H-MC       AR         229350.0      2
X-MC       XY25       225864.0      2
H-MC       XY24       206847.0      1
X-LC       XY24       203380.0      2
           X40N       202391.0      2
X-SC       X40        200560.0      2
M-SC       XR         186912.0      2
X-LC       XY25       135017.0      3
M-SC       AR         132937.0      2
M-LC       XR         109463.0      1
L-SC       OX40N       93757.0      1
           XR          87209.0      1
H-LC       AR          84123.0      1
L-MC       XR          51961.0      1
M-MC       XY25        46805.0      1
L-LC       XY25        24167.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
